# Case Study 2
---
<div style="margin: -10px 0 20px 0"><small>Author: Daniel Kończyk (18208152)</small></div>

This case study focuses on implementing a community-finding algorithm and comparing it's quality with one from the networkx package. I've decided to implement a `ratio-cut` algorithm with an option to do recursive cuts, from which the best one would be chosen based on modularity scores.

The study uses the following software:
* Python v3.7
* matplotlib package v3.1
* numpy package v1.16
* networkx package v2.3
* pandas package v0.24
* scipy package 1.3

The notebook has been executed in the following environment:
* Linux and macOS (PC and laptop)
* 16GB RAM
* 4-core CPU

This is the most memory-intensive Case Study and can use a lot of RAM. I tried to choose a network that wouldn't cause usage of more than 8GB, but I did not measure it precisely. I wanted to use sparse matrices as in other places, but unfortunately the eigenvectors functions in scipy, mainly `scipy.sparse.linalg.eigsh` turned out unreliable and I didn't have time to investigate further and went back to numpy, which requires a dense input.

We start the notebook with importing all required packages and modules:

In [1]:
%matplotlib inline
import networkx as nx
import numpy as np
import scipy as sp
import itertools
import datetime
import time
import pandas as pd
from IPython.display import display_html 
from networkx.algorithms.community import greedy_modularity_communities, LFR_benchmark_graph

## Utility functions
---

To measure the quality of communities I use the following modularity function, which is taken straight from lecture notes. This function is also used in the partitioning algorithm to choose the best partition

Input:  
`A` - numpy array representing a given network  
`partitions` - list of partitions of the graph corresponding to A  

Returns:  
*modularity measure* - a single float numer

In [2]:
def modularity(A, partitions):
    n, m = A.shape
    # 2m - 2 *sum of edges
    deg = A.sum()
    
    # matrix with expected fraction of within-community edges 
    # for a random graph same degree distribution 
    B = np.outer(A.sum(axis=1), A.sum(axis=1))/deg
    # matrix with partition masks
    n, m = A.shape
    X = np.zeros((len(partitions),n))
    for i,p in enumerate(partitions):
        X[i,p]=1
     
    return np.trace((X.dot(A-B)).dot(X.T))/deg

To compare communities produced by different algorithms, I've decided to use *Rand Index*, which has a nice vectorized implementation by computing the complement

Input:  
`part1` - partitioning done by the first algorithm  
`part2` - partitioning done by the second algorithm  
`n` - numer of nodes in the related graph  

Returns:  
*Rand Index value* - a single float numer

In [3]:
def rand_index(part1, part2, n):
    # create a matrix that lists all the pairs (twice) 
    # and indicates whether or not the nodes belong to the same partition
    def pairs(part):
        x = np.zeros(n, dtype=int)
        for i,p in enumerate(part):
            x[p] = i+1
        x = abs(np.array([x]).T - x)   
        x[x > 0] = 1
        
        return x
    
    group1 = pairs(part1)
    group2 = pairs(part2)
    
    return 1 - abs(group1-group2).sum()/(n*(n-1))    

*Ratio cut* is based on analyzing the eigevector corresponding to the second smallest eigenvalue, called *Fiedler*. This function computes and returns this vector.

Input:  
`A` - numpy array representing a given network  

Returns:  
*1d eigenvector* - the Fiedler eigenvector

In [4]:
def get_fiedler(A):
    n, m = A.shape
    # create sparse diagonal degree matrix from A
    diags = A.sum(axis=1)
    D = sp.sparse.spdiags(diags.flatten(), [0], m, n)
    # create Laplacian matrix
    L = D-A
    # compute eigenvectors of the Laplacian
    # since Laplacian is symmetric, use eigh for better performance
    # data is returned in sorted order (by eigenvalues), so no need to sort 
    _, eigen_vec = np.linalg.eigh(L)
    
    return eigen_vec[:,1].A1

## Ratio cut
---

The following function encapsulates the core of the algorithm - it performs a bipartitioning of the graph based on the sorted Fielder eigenvector, which is then traversed from the highest to the lowest element and the ratio cut is computed. The whole vector is checked and the best ratio and best cut are returned to the caller.

Input:  
`A` - numpy array representing a given network  

Returns:  
*a tuple* - best ratio cut value and actual two partions the network was divided to

In [5]:
def ratio_cut(A):
    # sort cut vector by
    cut_vec = np.argsort(get_fiedler(A))
    # create a helper matrix to improve performance of edge counts in partitions
    # rows and columns of original matrix are sorted according to cut vector
    # this allows standard, continuous numpy indexing instead of advanvced one, which copies data
    E = A[:,cut_vec][cut_vec]
    # remove self loops
    np.fill_diagonal(E, 0)

    best_cut, best_ratio = None, None
    all_edges, in_edges = 0, 0

    # run a loop and find the best cut 
    for i in range(1,cut_vec.size):
        # use E matrix to easily count all/inward/cut edges for a set or nodes
        all_edges += E[i-1].sum()
        in_edges += 2 * E[i-1,0:i-1].sum()
        cut_off_edges = all_edges - in_edges
        # compute ratio cut according to formula
        ratio_cut = cut_off_edges / min(i, cut_vec.size - i)
        
        # save the best cut and ratio
        if best_ratio is None or best_ratio >= ratio_cut:
            best_cut, best_ratio = i, ratio_cut
   
    return best_ratio, [cut_vec[0:best_cut], cut_vec[best_cut:]]

Since many datasets don't have continuous indices, a simple function is provided to translate between indices (since networkx resets original indices into a continuous space when exporting). This is mostly for reporting, as we may want to see both metrics, but also actual partitions with original node labels:

In [6]:
def translate(nodes, indices):
    return np.nonzero(indices[:,None] == nodes)[1]

This is the main function that runs the partitioning. It's fairly simple in a sense, that it strictly makes $2^N$ cuts and exits when a single element partition is created, since it can't be divided any further. There are more limitations, which will be discussed in the summary.

The function simply iterates until the depth limit is reached and cuts currently held partitioning one more time. It starts with 1 partition and divides into to 2, then enterd the loop having 2 partitions and further divides them into 4 and so on. While the loop runs, a modularity for the partitioning is recorded, so that only the best partitioning is returned (the one with highest modularity).

Input:  
`G` - graph to be partitioned  
`max_depth` - how many cutting iterations to perform, default is 1 (bipartition)

Returns:  
*list of partitions* - this is a list of numpy arrays, where each array is one of the partitions of the best cut

In [7]:
def partition(G, max_depth=1):
    
    npa = nx.to_numpy_array(G)
    best_cut, best_mod = None, None
    partitions = [npa]
    nodes = list(G.nodes())
    mappings = np.array([nodes])
    
    while max_depth > 0:
        if not (np.array([len(x) for x in partitions]) > 1).all():
            return best_cut
        
        raw_cut = list(itertools.chain(*[ratio_cut(p)[1] for p in partitions]))
        cut = [mappings[int(i/2)][k] for i,k in enumerate(raw_cut)]
        t_cut = [translate(nodes, c) for c in cut]
        mod = modularity(npa, t_cut)
        if best_mod is None or best_mod < mod:
            best_cut, best_mod = cut, mod
        
        subgraphs = [G.subgraph(c) for c in cut]
        mappings = [np.array(list(s.nodes())) for s in subgraphs]
        partitions = [nx.to_numpy_array(s) for s in subgraphs]
        max_depth -= 1
        
    return best_cut

## Tests and comparisons

This algorithm often performs bad on graphs with lots of tiny communities due to its handling of singular partitions. Still, I thought it would be good to include an example from the real world networks to see exactly what is happening. 

---
#### Real World Network
I chose `ca-GrQc` from https://snap.stanford.edu/data/ca-GrQc.html which was fairly small to fit in the memory, most of the other undirected graphs on SNAP were just too large fo this version of the algorithm.

The code captures modularity of the computed partitions and a rand index between networkx algorithm and my ratio cut:

In [8]:
s = time.perf_counter()
G = nx.read_edgelist("ca-GrQc.txt", create_using=nx.Graph(), nodetype=int)
p1 = [translate(G.nodes(), np.array(list(p))) for p in greedy_modularity_communities(G)]
p2 = [translate(G.nodes(), p) for p in partition(G,16)]

A = nx.to_numpy_array(G)
nx_modularity = modularity(A,p1)
rc_modularity = modularity(A,p2)
real_rand_index = rand_index(p1, p2, A.shape[1])
print("Computation time: {}".format(datetime.timedelta(seconds=int(time.perf_counter()-s))))

Computation time: 0:01:03


Now we can display and discuss the results:

In [9]:
df = pd.DataFrame([
    ["Community Quality (modularity)", nx_modularity, rc_modularity],
    ["Number of partitions", len(p1), len(p2)],
    ["Largest partition", max([len(p) for p in p1]),max([len(p) for p in p2])],
    ["Smallest partition", min([len(p) for p in p1]),min([len(p) for p in p2])],
    ["Communities comparison (rand index, same for both)", real_rand_index, real_rand_index],
    ],
    columns=["Metric", "networkx", "ratio-cut"])

df_style = (df.style
    .set_table_attributes("style='display:inline'")
    .set_caption("Communities comparison (G: nodes {})".format(len(G)))
    .hide_index()        
    .set_precision(7))
display_html(df_style._repr_html_(), raw=True)

Metric,networkx,ratio-cut
Community Quality (modularity),0.8118517,0.0002070131
Number of partitions,426,4
Largest partition,1039,5238
Smallest partition,1,1
"Communities comparison (rand index, same for both)",0.06305564,0.06305564


It is quite clear that ratio-cut did not uncover any substantial communities, with alsmot all the nodes sitting in one partition. One of the reason is that the small (2 nodes, exists in networkx partitioning too) partition was uncovered very early and the algorithm had to exit since there was nothing to further cut in this part. The difference is striking and as I mentioned before, some refinements would be needed to make it behave in more intelligent way. Some of these ideas will be discussed in the summary

#### Small synthetic graph
Next test will be run on a small, 500 nodes synthetic graph generated by networkx LFR benchmark, collecting the same data as the previous test, but also checking how well the finding matches the actual communities in the graph:

In [10]:
s = time.perf_counter()
G = LFR_benchmark_graph(n=500, tau1=3, tau2=1.5, mu=0.1, average_degree=5, min_community=60, seed=10)
p1 = [translate(G.nodes(), np.array(list(p))) for p in greedy_modularity_communities(G)]
p2 = [translate(G.nodes(), p) for p in partition(G,16)]
p3 = np.unique([list(G.nodes[v]['community']) for v in G])

A = nx.to_numpy_array(G)
nx_modularity = modularity(A,p1)
rc_modularity = modularity(A,p2)
lfr_modularity = modularity(A,p3)
rand_index_p1p2 = rand_index(p1, p2, A.shape[1])
rand_index_p1p3 = rand_index(p1, p3, A.shape[1])
rand_index_p2p3 = rand_index(p2, p3, A.shape[1])
print("Computation time: {}".format(datetime.timedelta(seconds=int(time.perf_counter()-s))))

Computation time: 0:00:00


In [11]:
df = pd.DataFrame([
    ["Community Quality (modularity)", nx_modularity, rc_modularity],
    ["Number of partitions", len(p1), len(p2)],
    ["Largest partition", max([len(p) for p in p1]),max([len(p) for p in p2])],
    ["Smallest partition", min([len(p) for p in p1]),min([len(p) for p in p2])],
    ["Found communities comparison (rand index, same for both)", rand_index_p1p2, rand_index_p1p2],
    ["Found communities comparison with actual data (rand index, different for both)", rand_index_p1p3, rand_index_p2p3],
    ],
    columns=["Metric", "networkx", "ratio-cut"])

df_style = (df.style
    .set_table_attributes("style='display:inline;width:100%'")
    .set_caption("Communities comparison (G: nodes {}, communities {}, \
                 largest partition {}, smallest partition {})".format(len(G), len(p3), 
                                                                      max([len(p) for p in p3]), 
                                                                      min([len(p) for p in p3])))
    .hide_index()        
    .set_precision(7))
display_html(df_style._repr_html_(), raw=True)

Metric,networkx,ratio-cut
Community Quality (modularity),0.6655362,0.659621
Number of partitions,4,4
Largest partition,147,148
Smallest partition,102,101
"Found communities comparison (rand index, same for both)",0.9779238,0.9779238
"Found communities comparison with actual data (rand index, different for both)",0.9934669,0.971503


For this synthetic graph, the outlook is much different. Both algorithms partitioned the graph almost perfectly as seen in rand index both between the algorithms and the actual communities in the graph.

#### Large synthetic graph
Next test will be run on a larger, 5000 nodes synthetic graph generated by networkx LFR benchmark, collecting the same data as the previous test:

In [12]:
s = time.perf_counter()
G = LFR_benchmark_graph(n=5000, tau1=2.8, tau2=1.5, mu=0.15, average_degree=25, min_community=80, seed=10)
p1 = [translate(G.nodes(), np.array(list(p))) for p in greedy_modularity_communities(G)]
p2 = [translate(G.nodes(), p) for p in partition(G,16)]
p3 = np.unique([list(G.nodes[v]['community']) for v in G])

A = nx.to_numpy_array(G)
nx_modularity = modularity(A,p1)
rc_modularity = modularity(A,p2)
lfr_modularity = modularity(A,p3)
rand_index_p1p2 = rand_index(p1, p2, A.shape[1])
rand_index_p1p3 = rand_index(p1, p3, A.shape[1])
rand_index_p2p3 = rand_index(p2, p3, A.shape[1])
print("Computation time: {}".format(datetime.timedelta(seconds=int(time.perf_counter()-s))))

Computation time: 0:02:18


In [13]:
df = pd.DataFrame([
    ["Community Quality (modularity)", nx_modularity, rc_modularity],
    ["Number of partitions", len(p1), len(p2)],
    ["Largest partition", max([len(p) for p in p1]),max([len(p) for p in p2])],
    ["Smallest partition", min([len(p) for p in p1]),min([len(p) for p in p2])],
    ["Found communities comparison (rand index, same for both)", rand_index_p1p2, rand_index_p1p2],
    ["Found communities comparison with actual data (rand index, different for both)", rand_index_p1p3, rand_index_p2p3],
    ],
    columns=["Metric", "networkx", "ratio-cut"])

df_style = (df.style
    .set_table_attributes("style='display:inline;width:100%'")
    .set_caption("Communities comparison (G: nodes {}, communities {}, \
                 largest partition {}, smallest partition {})".format(len(G), len(p3), 
                                                                      max([len(p) for p in p3]), 
                                                                      min([len(p) for p in p3])))
    .hide_index()        
    .set_precision(7))
display_html(df_style._repr_html_(), raw=True)

Metric,networkx,ratio-cut
Community Quality (modularity),0.6215929,0.2060774
Number of partitions,8,16
Largest partition,909,4079
Smallest partition,96,1
"Found communities comparison (rand index, same for both)",0.4323737,0.4323737
"Found communities comparison with actual data (rand index, different for both)",0.9617047,0.4477797


This time ratio-cut found more communities that were actually there and as before had to stop with 1-node partition. It is quite often the case, that ratio cut partitions the graph in quite uneven halves, resulting in a very large community at the end of the process. This is a potential for another change, where such disproportion would be detected and extra cut scenarios deployed.

## Summary

Ratio cut is often a good choice for an initial bipartinioning of the graph, but seems to sometimes fall short in a recursive scenario, requiring much more tuning than doing only recursive cuts. Despite the limitations, it is still able to find communities in some of the graphs, which seems quite surprising given it's rather simple structure. With a few enhancements, the quality of the algorithm could be vastly improved. I've already noted a few things I will most likely re-visit in the future:
* Don't stop if one of the partitions is small (1 node), but cut the other ones. This will result in a partitioning not bounded by $2^N$ and could uncover extra communities. The algorithm very often quits early, losing all that potential
* Perform ratio cuts in parallel - this could considerably speed up the algorithm for larger graphs
* Re-visit using sparse matrices - the current cost of computations is extremely high, but I couldn't find a reliable to compute eigenvalues on the sparse matrix